In [5]:
# %pip install openai
%pip install python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import json
import openai
from dotenv import load_dotenv

# Load .env if available
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Game variables
player_name = "Ihno"
save_file = "game_state.json"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# Story generation using ChatGPT
def generate_story(context, player_input, difficulty):
    messages = [
        {
            "role": "system",
            "content": (
                f"You are a text-based fantasy adventure game. "
                f"Difficulty is {['Easy', 'Medium', 'Hard'][difficulty]}. "
                f"Be creative, descriptive, and immersive. Respond with only the next part of the story, not instructions."
            ),
        },
        {"role": "user", "content": context + f"\n{player_name}: {player_input}"},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.9,
        max_tokens=250,
    )
    return response.choices[0].message["content"].strip()

# Difficulty adjustment
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 2)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 0)
    return difficulty

# Save/load/delete
def save_game():
    with open(save_file, "w", encoding="utf-8") as f:
        json.dump({
            "context": context,
            "stats": player_stats,
            "inventory": inventory,
            "difficulty": difficulty,
            "memory": game_memory
        }, f, indent=4)

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
        print("🗑️ Previous save data deleted.")
    else:
        print("⚠️ No save file found to delete.")

# Start menu
game_running = False
while not game_running:
    choice = input("\n💾 Load game (l), 🆕 New game (n), 🗑️ Delete save (d), or ❌ Quit (q)? ").strip().lower()
    if choice == 'l':
        if os.path.exists(save_file):
            with open(save_file, "r", encoding="utf-8") as f:
                game_state = json.load(f)
                context = game_state["context"]
                player_stats = game_state["stats"]
                inventory = game_state["inventory"]
                difficulty = game_state["difficulty"]
                game_memory = game_state["memory"]
            print("✅ Game loaded.")
            game_running = True
        else:
            print("⚠️ No saved game found.")
    elif choice == 'n':
        context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
        game_memory = [context]
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch"]
        difficulty = 1
        print("🆕 New game started.")
        game_running = True
    elif choice == 'd':
        delete_save()
    elif choice == 'q':
        print("🛑 Exiting game.")
        exit()
    else:
        print("❓ Invalid choice.")

# --- GAME LOOP ---
print("\n🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}")

    player_input = input("\n💬 What do you do? (type 'save', 'quit' or your action): ").strip()

    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        print("💾 Game saved.")
        continue

    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    # Context for model
    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)

    context += f"\n{player_name}: {player_input}\n{new_story}"
    game_memory.append(new_story)

    # Log story
    with open("story_log.txt", "a", encoding="utf-8") as log:
        log.write(f"{player_name}: {player_input}\n{new_story}\n\n")


🆕 New game started.

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


Draft 2


In [8]:
import os
import json
import openai
from dotenv import load_dotenv

# Load .env if available
load_dotenv()

# Create OpenAI client with your API key
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Game variables
player_name = "Ihno"
save_file = "game_state.json"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# Story generation using ChatGPT
def generate_story(context, player_input, difficulty):
    messages = [
        {
            "role": "system",
            "content": (
                f"You are a text-based fantasy adventure game. "
                f"Difficulty is {['Easy', 'Medium', 'Hard'][difficulty]}. "
                f"Be creative, descriptive, and immersive. Respond with only the next part of the story, not instructions."
            ),
        },
        {
            "role": "user",
            "content": context + f"\n{player_name}: {player_input}",
        },
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.9,
        max_tokens=250,
    )

    return response.choices[0].message.content.strip()

# Difficulty adjustment
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 2)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 0)
    return difficulty

# Save/load/delete
def save_game():
    with open(save_file, "w", encoding="utf-8") as f:
        json.dump({
            "context": context,
            "stats": player_stats,
            "inventory": inventory,
            "difficulty": difficulty,
            "memory": game_memory
        }, f, indent=4)

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
        print("🗑️ Previous save data deleted.")
    else:
        print("⚠️ No save file found to delete.")

# Start menu
game_running = False
while not game_running:
    choice = input("\n💾 Load game (l), 🆕 New game (n), 🗑️ Delete save (d), or ❌ Quit (q)? ").strip().lower()
    if choice == 'l':
        if os.path.exists(save_file):
            with open(save_file, "r", encoding="utf-8") as f:
                game_state = json.load(f)
                context = game_state["context"]
                player_stats = game_state["stats"]
                inventory = game_state["inventory"]
                difficulty = game_state["difficulty"]
                game_memory = game_state["memory"]
            print("✅ Game loaded.")
            game_running = True
        else:
            print("⚠️ No saved game found.")
    elif choice == 'n':
        context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
        game_memory = [context]
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch"]
        difficulty = 1
        print("🆕 New game started.")
        game_running = True
    elif choice == 'd':
        delete_save()
    elif choice == 'q':
        print("🛑 Exiting game.")
        exit()
    else:
        print("❓ Invalid choice.")

# --- GAME LOOP ---
print("\n🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}")

    player_input = input("\n💬 What do you do? (type 'save', 'quit' or your action): ").strip()

    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        print("💾 Game saved.")
        continue

    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    # Context for model
    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)

    context += f"\n{player_name}: {player_input}\n{new_story}"
    game_memory.append(new_story)

    # Log story
    with open("story_log.txt", "a", encoding="utf-8") as log:
        log.write(f"{player_name}: {player_input}\n{new_story}\n\n")


🆕 New game started.

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

Draft 3

In [5]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load GPT-2 tokenizer and model
model_name = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Game variables
player_name = "Ihno"
save_file = "game_state.json"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# Story generation using GPT-2
def generate_story(context, player_input, difficulty):
    prompt = (
        f"You are a text-based fantasy adventure game.\n"
        f"Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}\n"
        f"{context}\n"
        f"{player_name}: {player_input}\nGame:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    story = output_text.split("Game:")[-1].strip()
    return story

# Difficulty adjustment
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 2)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 0)
    return difficulty

# Save/load/delete
def save_game():
    with open(save_file, "w", encoding="utf-8") as f:
        json.dump({
            "context": context,
            "stats": player_stats,
            "inventory": inventory,
            "difficulty": difficulty,
            "memory": game_memory
        }, f, indent=4)

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
        print("🗑️ Previous save data deleted.")
    else:
        print("⚠️ No save file found to delete.")

# Start menu
game_running = False
while not game_running:
    choice = input("\n💾 Load game (l), 🆕 New game (n), 🗑️ Delete save (d), or ❌ Quit (q)? ").strip().lower()
    if choice == 'l':
        if os.path.exists(save_file):
            with open(save_file, "r", encoding="utf-8") as f:
                game_state = json.load(f)
                context = game_state["context"]
                player_stats = game_state["stats"]
                inventory = game_state["inventory"]
                difficulty = game_state["difficulty"]
                game_memory = game_state["memory"]
            print("✅ Game loaded.")
            game_running = True
        else:
            print("⚠️ No saved game found.")
    elif choice == 'n':
        context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
        game_memory = [context]
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch"]
        difficulty = 1
        print("🆕 New game started.")
        game_running = True
    elif choice == 'd':
        delete_save()
    elif choice == 'q':
        print("🛑 Exiting game.")
        exit()
    else:
        print("❓ Invalid choice.")

# --- GAME LOOP ---
print("\n🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}")

    player_input = input("\n💬 What do you do? (type 'save', 'quit' or your action): ").strip()

    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        print("💾 Game saved.")
        continue

    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    # Context for model
    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)

    context += f"\n{player_name}: {player_input}\n{new_story}"
    game_memory.append(new_story)

    # Log story
    with open("story_log.txt", "a", encoding="utf-8") as log:
        log.write(f"{player_name}: {player_input}\n{new_story}\n\n")


🆕 New game started.

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.
Ihno: Helle sir.
Ihno Awakenings
Level Design: Not too difficult
Control: A-C
Graphics: OK
Soundtrack: OK
Ihno: You have awoken in a forest. There is a figure who asks you to guide him as you seek his treasure. Help him reach the treasures of the lands.
How to Play: Go to the nearest map marker in the world and navigate the map. Take turns playing as Helle and guiding him as he explores.
A-C-C-A-B-D-A
A-C-C-A-B-D-A
Game Features:
Ihno features a number of optional actions.
There are optional actions to move the map marker

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium

📖 Story so far:
Ihno awakens in a dark forest

Draft gpt 2, clear prompt

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.eval()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Game State
player_name = "Ihno"
inventory = ["torch"]
stats = {"health": 100, "strength": 10, "gold": 5}
difficulty = "Medium"
story_context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."

# Prompt formatting
def build_prompt(context, player_input):
    return (
        "This is a fantasy text adventure. Continue the story based on the player's actions.\n"
        f"Story so far:\n{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )

# Generate story continuation
def generate_response(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        inputs["input_ids"],
        max_length=inputs["input_ids"].shape[1] + max_length,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.8,
        top_k=50,
        top_p=0.95,
    )
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text[len(prompt):].strip().split("\n")[0]

# Game loop
print("🆕 New game started.\n")
print("🌲 Welcome to the AI Dungeon! 🌲\n")
print(f"📖 Story so far:\n{story_context}\n")
print(f"🧍 {player_name}'s Inventory: {inventory}")
print(f"❤️ Stats: {stats} | 🎯 Difficulty: {difficulty}\n")

while True:
    player_input = input(f"{player_name}: ")

    if player_input.lower() in ["quit", "exit"]:
        print("🛑 Game ended. Thanks for playing!")
        break

    prompt = build_prompt(story_context, player_input)
    narration = generate_response(prompt)

    # Update story context
    story_context += f"\n{player_name}: {player_input}\nNarrator: {narration}"

    # Show updated story
    print(f"\n📖 {narration}\n")
    print(f"🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {stats} | 🎯 Difficulty: {difficulty}\n")


🆕 New game started.

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium



The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



📖 A strange figure starts to search for me.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium


📖 The strange figure looks at me.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium


📖 I enter the forest.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium


📖 I see a monster, and I run away.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium


📖 I see a monster, and I run away.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Medium



Draft full code:

In [6]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# --- CONFIGURATION ---
model_name = "gpt2-medium"
save_file = "game_state.json"
log_file = "story_log.txt"

# --- LOAD MODEL & TOKENIZER ---
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# --- INITIALIZE GAME STATE ---
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- GENERATE STORY ---
def generate_story(context, player_input, difficulty):
    prompt = (
        "This is a fantasy adventure story.\n"
        f"Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}\n"
        f"{context}\n"
        f"{player_name}: {player_input}\nNarrator:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=400,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Strip the prompt part off to get only new generation
    generated = output_text[len(prompt):].strip()

    # Fallback if model echoes the prompt too closely
    if not generated:
        generated = output_text.split(player_input)[-1].strip()

    return generated


# --- DIFFICULTY LOGIC ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 2)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 0)
    return difficulty

# --- SAVE/LOAD ---
def save_game():
    with open(save_file, "w", encoding="utf-8") as f:
        json.dump({
            "context": context,
            "stats": player_stats,
            "inventory": inventory,
            "difficulty": difficulty,
            "memory": game_memory
        }, f, indent=4)
    print("💾 Game saved.")

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
        print("🗑️ Previous save data deleted.")
    else:
        print("⚠️ No save file found to delete.")

# --- START MENU ---
game_running = False
while not game_running:
    choice = input("\n💾 Load game (l), 🆕 New game (n), 🗑️ Delete save (d), or ❌ Quit (q)? ").strip().lower()
    if choice == 'l':
        if os.path.exists(save_file):
            with open(save_file, "r", encoding="utf-8") as f:
                game_state = json.load(f)
                context = game_state["context"]
                player_stats = game_state["stats"]
                inventory = game_state["inventory"]
                difficulty = game_state["difficulty"]
                game_memory = game_state["memory"]
            print("✅ Game loaded.")
            game_running = True
        else:
            print("⚠️ No saved game found.")
    elif choice == 'n':
        context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
        game_memory = [context]
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch"]
        difficulty = 1
        print("🆕 New game started.")
        game_running = True
    elif choice == 'd':
        delete_save()
    elif choice == 'q':
        print("🛑 Exiting game.")
        exit()
    else:
        print("❓ Invalid choice.")

# --- GAME LOOP ---
print("\n🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard'][difficulty]}")

    player_input = input("\n💬 What do you do? (type 'save', 'quit' or your action): ").strip()

    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        continue

    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)

    context += f"\n{player_name}: {player_input}\n{new_story}"
    game_memory.append(new_story)

    # Log to file
    with open(log_file, "a", encoding="utf-8") as log:
        log.write(f"{player_name}: {player_input}\n{new_story}\n\n")


OSError: gpt3 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

Draft openai gpt3

In [1]:
import os
import json
import time
from dotenv import load_dotenv
import openai
from openai import OpenAI, RateLimitError

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")

client = OpenAI(api_key=api_key)

# --- CONFIGURATION ---
model_name = "gpt-3.5-turbo"
save_file = "game_state.json"
log_file = "story_log.txt"

# --- GAME STATE ---
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY LOGIC ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )

    max_retries = 5  # Increase the number of retries
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=0.9,
                top_p=0.95
            )
            return response.choices[0].message.content.strip()

        except RateLimitError as e:
            wait_time = 2 ** attempt  # Exponential backoff
            print(f"⚠️ Rate limited. Retrying in {wait_time} seconds... Attempt {attempt+1}/{max_retries}")
            time.sleep(wait_time)

    raise Exception(f"❌ Rate limit exceeded after {max_retries} attempts.")

# --- SAVE/LOAD ---
def save_game():
    with open(save_file, "w", encoding="utf-8") as f:
        json.dump({
            "context": context,
            "stats": player_stats,
            "inventory": inventory,
            "difficulty": difficulty,
            "memory": game_memory
        }, f, indent=4)
    print("💾 Game saved.")

def delete_save():
    if os.path.exists(save_file):
        os.remove(save_file)
        print("🗑️ Previous save data deleted.")
    else:
        print("⚠️ No save file found to delete.")

# --- START MENU ---
game_running = False
while not game_running:
    choice = input("\n💾 Load game (l), 🆕 New game (n), 🗑️ Delete save (d), or ❌ Quit (q)? ").strip().lower()
    if choice == 'l':
        if os.path.exists(save_file):
            with open(save_file, "r", encoding="utf-8") as f:
                game_state = json.load(f)
                context = game_state["context"]
                player_stats = game_state["stats"]
                inventory = game_state["inventory"] 
                difficulty = game_state["difficulty"]
                game_memory = game_state["memory"]
            print("✅ Game loaded.")
            game_running = True
        else:
            print("⚠️ No saved game found.")
    elif choice == 'n':
        context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
        game_memory = [context]
        player_stats = {"health": 100, "strength": 10, "gold": 5}
        inventory = ["torch"]
        difficulty = 1
        print("🆕 New game started.")
        game_running = True
    elif choice == 'd':
        delete_save()
    elif choice == 'q':
        print("🛑 Exiting game.")
        exit()
    else:
        print("❓ Invalid choice.")

# --- GAME LOOP ---
print("\n🌲 Welcome to the AI Dungeon! 🌲")
while True:
    print("\n📖 Story so far:")
    print(context)
    print(f"\n🧍 {player_name}'s Inventory: {inventory}")
    print(f"❤️ Stats: {player_stats} | 🎯 Difficulty: {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}")

    player_input = input("\n💬 What do you do? (type 'save', 'quit' or your action): ").strip()

    if player_input.lower() == 'quit':
        print("🛑 Game ended. Thanks for playing!")
        break
    elif player_input.lower() == 'save':
        save_game()
        continue

    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    try:
        new_story = generate_story(recent_context, player_input, difficulty)
        context += f"\n{player_name}: {player_input}\n{new_story}"
        game_memory.append(new_story)

        # Log to file
        with open(log_file, "a", encoding="utf-8") as log:
            log.write(f"{player_name}: {player_input}\n{new_story}\n\n")
    except Exception as e:
        print(f"❌ Error during story generation: {e}")
        break


🆕 New game started.

🌲 Welcome to the AI Dungeon! 🌲

📖 Story so far:
Ihno awakens in a dark forest. A mysterious figure approaches.

🧍 Ihno's Inventory: ['torch']
❤️ Stats: {'health': 100, 'strength': 10, 'gold': 5} | 🎯 Difficulty: Easy
⚠️ Rate limited. Retrying in 1 seconds... Attempt 1/5
⚠️ Rate limited. Retrying in 2 seconds... Attempt 2/5
⚠️ Rate limited. Retrying in 4 seconds... Attempt 3/5
⚠️ Rate limited. Retrying in 8 seconds... Attempt 4/5
⚠️ Rate limited. Retrying in 16 seconds... Attempt 5/5
❌ Error during story generation: ❌ Rate limit exceeded after 5 attempts.
